<div class="alert alert-block alert-info">
This script merges all eeg files for a session and splits them back according to session timestamps. The Jupyer Notebook was made for a single
session, and the Python script is better for multiple sessions.
</div>

In [40]:
import numpy as np
import os
import csv
import json
import argparse

## Variables
These variables may change:

In [41]:
animal_name='JC258'
date='20210626'

basedir = "/mnt/adata9/"
mbasedir="/adata_pool/merged/"+animal_name+'-'+date+'/'

These variables do not change:

In [42]:
sample_rate_res_old=24000

sample_rate_whl=39.0625
sample_rate_res=20000

sample_rate_eegh=5000

downsampled_res=sample_rate_res/sample_rate_res_old

## Import session metadata:

In [43]:
session_metadata = {}
with open('session_metadata.csv', mode='r', encoding='utf-8') as file:
    reader = csv.DictReader(file, delimiter=";")    
    for line in reader:
        session_id = line.pop('session_id')
        session_metadata[session_id] = line

In [44]:
basename = animal_name+'-'+date
print('Metadata for',basename)
session_metadata[basename]

Metadata for JC258-20210626


{'num_tetr': '24',
 'last_pfc_left': '8',
 'last_pfc_right': '15',
 'session_names': "['training1','intersleep','training2']",
 'session_idx': '[[1],[2],[3]]',
 'reward_arms': '4,8',
 'rewards': 'C,H'}

In [45]:
num_tetrodes = int(session_metadata[basename]['num_tetr'])

session_idx = json.loads(session_metadata[basename]['session_idx'])

session_names_str = session_metadata[basename]['session_names']
session_names_str = session_names_str.replace("'",'"') # the single quotes aren't being read in JSON, but I can't use " in the csv because it's a str delimiter
session_names = json.loads(session_names_str)

reward_arms_str = session_metadata[basename]['reward_arms']
reward_arms = np.array(list(map(int, reward_arms_str.split(','))))

## Merge and split eegh files and export reward arm files

In [46]:
def find_files_with_eegh(directory):
    eegh_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if 'eegh' in file:
                eegh_files.append(os.path.join(root, file))
                
    return eegh_files

In [47]:
# basedir='D:\github/test/'
# mbasedir="D:\github/test\merged"+animal_name+'-'+date+'/'
if not os.path.isdir(mbasedir):
    os.makedirs(mbasedir)
directory_path_eegh = basedir+"eeg/"+animal_name+'/'+date+'/'

In [48]:
# find all eegh files in a session
eegh_files = sorted(find_files_with_eegh(directory_path_eegh)) # sort the filenames numerically
print('Found eegh files:')
for file in eegh_files:
    print(file)

# initialize a merged eegh file and add the first file to it
eegh_merged=np.fromfile(eegh_files[0], dtype=np.int16)
# reshape the first file (1D to 2D array) so that each row corresponds to one tetrode
eegh_merged= eegh_merged.reshape(int(len(eegh_merged)/num_tetrodes),num_tetrodes)

# iterate over the other files and add them to the large merged file
for eegh_file_i in range(1,len(eegh_files)):
    eegh_t=np.fromfile(eegh_files[eegh_file_i], dtype=np.int16)
    eegh_t= eegh_t.reshape(int(len(eegh_t)/num_tetrodes),num_tetrodes)
    eegh_merged=np.append(eegh_merged,eegh_t,axis=0)

print(eegh_merged.shape)
    
# calculate the total length (num timestamps) of the merged eegh file
length_eegh_merged=eegh_merged.shape[0]

Found eegh files:
/mnt/adata9/eeg/JC258/20210626/JC258-20210626_01.eegh
/mnt/adata9/eeg/JC258/20210626/JC258-20210626_02.eegh
/mnt/adata9/eeg/JC258/20210626/JC258-20210626_03.eegh
(38609536, 24)


In [49]:
# load the session timestamps and downsample them
session_timestamps=np.loadtxt(basedir+"processing/"+animal_name+'/'+date+'/'+'session_shifts.txt')

session_timestamps=np.append([0],session_timestamps) # start the first timestamp at 0
session_timestamps_down=session_timestamps*downsampled_res

for session_idx_i in range(len(session_idx)):
    # generate reward arms files for the training session files
    if session_names[session_idx_i]=='training1' or session_names[session_idx_i]=='training2':
        np.savetxt(mbasedir+animal_name+'-'+date+'_'+session_names[session_idx_i]+'.reward_arms', reward_arms, fmt='%i',newline=" ")
    
    # cut the eegh files according to the session timestamps and write them to the merged folder
    start_cut=session_idx[session_idx_i][0]-1
    end_cut=session_idx[session_idx_i][-1]
    start_eegh=int(session_timestamps_down[start_cut]/sample_rate_res*sample_rate_eegh)
    end_eegh=int(session_timestamps_down[end_cut]/sample_rate_res*sample_rate_eegh)

    eegh_temp=eegh_merged[start_eegh:end_eegh,:]

    eegh_temp.tofile(mbasedir+animal_name+'-'+date+'_'+session_names[session_idx_i]+'.eegh')